In [78]:
import gurobipy as grb
import numpy as np
import itertools

In [79]:
def iterer(*args):
    return itertools.product(*[range(x_) for x_ in args])

In [84]:
class Optimizer:
    """
        Single (big) robot optimizer
    """
    actions = {
        "heaps": 6,
        "funny": 2,
        "disposal": 1,
        "base": 1
    }
    def __init__(self, **kvargs):
        self.model = grb.Model("eurobot")
        self.actions = {}
        
        # choose some certain actions
        # by default all from Optimizer.actions
        if "actions" in kvargs:
            try:
                for a in kvargs["actions"]:
                    self.actions[a] = Optimizer.actions[a]
            except:
                print("Error: wrong actions in kvargs")
        else:
            self.actions.update(Optimizer.actions)
        
        
        # N for total available actions (from them we can choose)
        self.N = sum(self.actions.values())
        # S for total maximal number of actions can be created by optimizer
        self.S = self.N - 3
        self.x = self.model.addVars(range(self.S), range(self.N), name = 'x', vtype=grb.GRB.BINARY)
        
        #define positions of all actions in x variable
        self.actions_iters = {}
        i = 0
        for a in self.actions:
            self.actions_iters[a] = list(range(i,i+self.actions[a]))
            i += self.actions[a]
        print(self.actions_iters)
        self.extra_conditions = []
       
        # set default Time Matrix
        self.createDefaultTimeMatrix()
        
        # create time constraint/objective
        self.createTimeConstraint()
        
        # create points contraint/objective
        self.createPointsConstraint()
        
        self.Time = 100.0
        self.Points = 220.0
    
        self.objective = "time"
        if "objective" in kvargs:
            if kvargs["objective"] in ["time","points","mixed"]:
                self.objective = kvargs["objective"]
        
    def resetConstraints(self):
        self.model.remove(self.model.getConstrs())
        
    def createDefaultTimeMatrix(self):
        ## Calculate time matrix
        # places with cubes
        heap_points = np.array([[54, 85, 0], [119, 30, 0], [150, 110, 0], [150, 190, 0], [119, 270, 0], [54, 215, 0]])

        # starting place of robot
        # to be redefined at the start of the algorithm
        base_point  = np.array([[15, 15, 0]])

        # disposal place 
        # TODO: add another side of field
        disposal_point = np.array([[10, 61, 0]])

        # funny action places
        funny_points = np.array([[10, 113, 0], [190, 10, 0]])

        action_points = np.concatenate((heap_points, funny_points, disposal_point, base_point))

        # time is simple distance / v
        v = 20

        self.time_travel = np.sum((action_points[np.newaxis,:]-action_points[:,np.newaxis]) ** 2, axis = 2)**0.5 / v
        
    def addDefaultConstraints(self):
        # unique action per 's'-tep
        self.model.addConstrs((self.x.sum(s, '*') == 1 for s in iterer(self.S)));
        
        # each action can be done only once
        self.model.addConstrs((self.x.sum('*', i) <= 1 for i in range(self.N)));
            
        # now max 3 heaps
        self.model.addConstr(grb.quicksum((self.x.sum('*',i) for i in iterer(self.actions["heaps"]))) <= 3 );
        
        # disposal after all 3 heaps picking
        for s, k in iterer(self.S,self.actions["heaps"]):
            self.model.addConstr( grb.quicksum( (self.x[s_,self.actions_iters["disposal"][0]] for s_ in range(s,self.S)) )
                                 >= self.x[s,k] );
        # starting from base point
        self.model.addConstr(self.x[0,self.actions_iters["base"][0]] == 1);
        
    def createTimeConstraint(self):
        time_travel_constr_gen = (self.x[s, k_1] * self.x[s+1, k_2] * self.time_travel[k_1, k_2] for  s,k_1, k_2 in 
                                  #[(0,9, 2)])
                                  iterer(self.S-1,self.N,self.N))
        left_part_time_1 = grb.quicksum(time_travel_constr_gen)

        self.left_part_time = left_part_time_1
    
    def createPointsConstraint(self):
        #TODO: rewrite with points array
        self.total_points = 45*grb.quicksum((self.x.sum('*',k) for k in self.actions_iters["heaps"])) + \
                                         30*self.x.sum('*',self.actions_iters["funny"][0]) + \
                                         55*self.x.sum('*',self.actions_iters["funny"][1]);
    def setObjective(self):
        """
            minimize "time"
            or
            maximize "points"
        """
        if self.objective=="time":
            self.model.addConstr(self.total_points == self.Points, 'points')
            self.model.setObjective(self.left_part_time, grb.GRB.MINIMIZE)
        elif self.objective=="points":
            self.model.addConstr(self.left_part_time <= self.Time, 'time')
            self.model.setObjective(self.total_points, grb.GRB.MAXIMIZE)
        elif self.objective=="mixed":
            self.model.addConstr(self.left_part_time <= self.Time, 'time')
            self.model.setObjective(self.total_points - self.left_part_time/200, grb.GRB.MAXIMIZE)
    
    def run(self):
        self.resetConstraints()
        self.addDefaultConstraints()
        self.setObjective()
        
        self.model.update()
        self.model.optimize()
        self.model.printAttr('x')

In [85]:
opt = Optimizer(objective="mixed")

{'heaps': [0, 1, 2, 3, 4, 5], 'funny': [6, 7], 'disposal': [8], 'base': [9]}


In [86]:
opt.run()

Optimize a model with 61 rows, 70 columns and 393 nonzeros
Model has 540 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 0 continuous, 70 integer (70 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [2e+00, 1e+01]
  Objective range  [3e+01, 6e+01]
  QObjective range [2e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
  QRHS range       [1e+02, 1e+02]
Modified 48 Q diagonals
Modified 48 Q diagonals
Presolve removed 15 rows and 22 columns
Presolve time: 0.47s
Presolved: 46 rows, 48 columns, 270 nonzeros
Presolved model has 360 quadratic objective terms
Variable types: 0 continuous, 48 integer (48 binary)

Root relaxation: objective 2.209842e+02, 97 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  220.98416    0   48          -  220.98416      -     -    0s

In [87]:
opt.left_part_time.getValue()

24.393213213192926

In [88]:
opt.total_points.getValue()

220.0

In [73]:
for x in opt.x.items():
    if abs(x[1].X - 1) < 0.1:
        print(x[0])

(0, 9)
(1, 1)
(2, 7)
(3, 2)
(4, 0)
(5, 8)
(6, 6)


In [74]:
for k,v in {'a':1}.items():
    print(k,v)

a 1


In [77]:
tuple(k for k in {"a":1,"b":2})

('a', 'b')

In [93]:
(3+1/2)*np.pi % np.pi

1.5707963267948966

In [94]:
a = [1,2,3]

In [95]:
b = a.copy()